In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential, layers, models
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tensorflow.keras.callbacks import EarlyStopping
import h5py as h5
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

In [2]:
y_df = pd.read_csv("C:\\Users\\Admin\\Documents\\MindSee\\data\\y_dataframe.csv")
y_train = y_df.iloc[:8859, 1:].to_numpy()
y_df.drop(columns = ["Unnamed: 0"], inplace = True)
X_train = np.load("C:\\Users\\Admin\\Documents\\MindSee\\data\\nsd_train_fmriavg_nsdgeneral_sub1.npy")
X_test = np.load("C:\\Users\\Admin\\Documents\\MindSee\\data\\nsd_test_fmriavg_nsdgeneral_sub1.npy")


C:\Users\Admin\AppData\Local\Temp\ipykernel_8244\3560020842.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  y_df = pd.read_csv("C:\\Users\\Admin\\Documents\\MindSee\\data\\y_dataframe.csv")


In [3]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [20]:
from tensorflow import keras
custom_layer=TransformerBlock(100,100,100)

In [28]:
X_scaled.shape

(8859, 15724)

In [29]:
multihead_1=Sequential([layers.Dense(100, input_shape = (15724,), activation = "relu"),
            custom_layer,
#             custom_layer,
            layers.Dense(100, activation = "relu"),
                  layers.Dense(100, activation = "relu"),
                    layers.Dropout(0.1),
                  layers.Dense(200, activation = "relu"),
                    layers.Dropout(0.5),
                  layers.Dense(500, activation = "relu"),
                    layers.Dropout(0.5),
                    layers.Dense(500, activation = "relu"),
                    layers.Dropout(0.5),
                    layers.Dense(1000, activation = "relu"),
                    layers.Dropout(0.5),
                    layers.Dense(2000, activation = "relu"),
                  layers.Dense(2637, activation = "sigmoid")])


#Compile
multihead_1.compile(loss = "binary_crossentropy", optimizer = "adam")

#ES
es = EarlyStopping(patience=20,restore_best_weights = True)

#Fit
history_mh = multihead_1.fit(X_scaled, y_train, validation_split=0.15, batch_size = 1, epochs = 120, callbacks=[es])

StagingError: Exception encountered when calling layer "transformer_block_6" (type TransformerBlock).

in user code:

    File "C:\Users\Admin\AppData\Local\Temp\ipykernel_8244\1284121884.py", line 14, in call  *
        attn_output = self.att(inputs, inputs)
    File "C:\Users\Admin\miniconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Admin\miniconda3\lib\site-packages\keras\layers\activation\softmax.py", line 103, in call
        return backend.softmax(inputs, axis=self.axis[0])

    IndexError: Exception encountered when calling layer "softmax" "                 f"(type Softmax).
    
    tuple index out of range
    
    Call arguments received by layer "softmax" "                 f"(type Softmax):
      • inputs=tf.Tensor(shape=(None, 100), dtype=float32)
      • mask=None


Call arguments received by layer "transformer_block_6" (type TransformerBlock):
  • inputs=tf.Ten

In [23]:
custom_layer.

ValueError: Weights for model sequential_11 have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.